In [1]:
using LinearAlgebra


In [2]:
params = [ 1.5e+09 1.6e+09 1.633 0 1.633 0 306.19 8 2 2 3 ]

nstress = Int(params[9]);
ndim    = Int(params[11]);

In [3]:
for i=1:3
    println(i)
end

1
2
3


In [4]:
params'

11×1 adjoint(::Matrix{Float64}) with eltype Float64:
   1.5e9
   1.6e9
   1.633
   0.0
   1.633
   0.0
 306.19
   8.0
   2.0
   2.0
   3.0

In [5]:
transpose(params)

11×1 transpose(::Matrix{Float64}) with eltype Float64:
   1.5e9
   1.6e9
   1.633
   0.0
   1.633
   0.0
 306.19
   8.0
   2.0
   2.0
   3.0

In [6]:
    lambda  = params[1];
    mu      = params[2];
    Aphi    = params[3];
    Bphi    = params[4];
    Apsi    = params[5];
    Bpsi    = params[6];
    Hc      = params[7];
    nstress = Int(params[9 ]);
    nisv    = Int(params[10]);
    ndim    = Int(params[11])

3

In [7]:
X = [-1 -1  1] / sqrt(3)

1×3 Matrix{Float64}:
 -0.57735  -0.57735  0.57735

In [8]:
A = rand(3,3)

3×3 Matrix{Float64}:
 0.00614745  0.225921  0.837185
 0.648393    0.951914  0.227544
 0.455461    0.282014  0.985972

In [9]:
eigvals(A)

3-element Vector{Float64}:
 -0.3087969312653249
  0.6631303718593321
  1.5896992590428736

In [10]:
eigvecs(A)

3×3 Matrix{Float64}:
  0.883025   0.240099  -0.424221
 -0.414373  -0.873224  -0.654541
 -0.220367   0.424066  -0.625789

In [11]:
B = [1 2 3;
     4 5 6]

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

In [12]:
function grad_basis_linear(xi)

    # derivatives of shape functions with respect to xi
    dN1_dxi = -0.125*(1-xi[2])*(1-xi[3]);
    dN2_dxi = -dN1_dxi;
    dN3_dxi = 0.125*(1+xi[2])*(1-xi[3]);
    dN4_dxi = -dN3_dxi;
    dN5_dxi = -0.125*(1-xi[2])*(1+xi[3]);
    dN6_dxi = -dN5_dxi;
    dN7_dxi = 0.125*(1+xi[2])*(1+xi[3]);
    dN8_dxi = -dN7_dxi;

    # derivatives of shape functions with respect to eta
    dN1_deta = -0.125*(1-xi[1])*(1-xi[3]);
    dN2_deta = -0.125*(1+xi[1])*(1-xi[3]);
    dN3_deta = -dN2_deta;
    dN4_deta = -dN1_deta;
    dN5_deta = -0.125*(1-xi[1])*(1+xi[3]);
    dN6_deta = -0.125*(1+xi[1])*(1+xi[3]);
    dN7_deta = -dN6_deta;
    dN8_deta = -dN5_deta;

    # derivatives of shape functions with respect to zeta
    dN1_dzeta = -0.125*(1-xi[1])*(1-xi[2]);
    dN2_dzeta = -0.125*(1+xi[1])*(1-xi[2]);
    dN3_dzeta = -0.125*(1+xi[1])*(1+xi[2]);
    dN4_dzeta = -0.125*(1-xi[1])*(1+xi[2]);
    dN5_dzeta = -dN1_dzeta;
    dN6_dzeta = -dN2_dzeta;
    dN7_dzeta = -dN3_dzeta;
    dN8_dzeta = -dN4_dzeta;

    # Populate dNdX
    dNdX = [dN1_dxi dN2_dxi dN3_dxi dN4_dxi dN5_dxi dN6_dxi dN7_dxi dN8_dxi;
            dN1_deta dN2_deta dN3_deta dN4_deta dN5_deta dN6_deta dN7_deta dN8_deta;
            dN1_dzeta dN2_dzeta dN3_dzeta dN4_dzeta dN5_dzeta dN6_dzeta dN7_dzeta dN8_dzeta];
    return dNdX
end

grad_basis_linear (generic function with 1 method)

In [13]:
    # Helper function: 3x3 identity matrix
    eye_mat = I(3);

    # params = [ lambda mu Aphi Bphi Apsi Bpsi Hc numips nstress nisv ndim ];
    lambda  = params[1]
    mu      = params[2]
    Aphi    = params[3]
    Bphi    = params[4]
    Apsi    = params[5]
    Bpsi    = params[6]
    Hc      = params[7]
    nstress = Int(params[9 ])
    nisv    = Int(params[10])
    ndim    = Int(params[11])

    # initialize
    isv      = zeros(nisv)
    Fp_el    = zeros(ndim, ndim)
    stresses = zeros(ndim, ndim, nstress)

    # Mapping from natural coordinates to physical coordinates (X -> x)
    X = [-1 -1  1] / sqrt(3)
dNdX = grad_basis_linear(X)

3×8 Matrix{Float64}:
 -0.0833333   0.0833333   0.0223291  …   0.311004   0.0833333  -0.0833333
 -0.0833333  -0.0223291   0.0223291     -0.0833333  0.0833333   0.311004
 -0.311004   -0.0833333  -0.0223291      0.0833333  0.0223291   0.0833333

In [14]:
coordsx = [ 0.0  0.0  0.0;
            1.0  0.0  0.0;
            1.0  1.0  0.0;
            0.0  1.0  0.0;
            0.0  0.0  1.0;
            1.0  0.0  1.0;
            1.0  1.0  1.0;
            0.0  1.0  1.0 ];
d = [   0                  ,   0                   ,     0                   ,
       -0.00121108996046273,   0.004706716693526170,     0                   ,
       -0.00592063407691252,   0.004095620071092810,     0                   ,
       -0.00470954411644980,  -0.000611096622433368,     0                   ,
        0                  ,   0                   ,     0.000574665721779025,
       -0.00121108996046273,   0.004706716693526170,     0.000574665721779095,
       -0.00592063407691252,   0.004095620071092810,     0.000574665721779159,
       -0.00470954411644980,  -0.000611096622433368,     0.000574665721779107 ];
Je = dNdX * coordsx

3×3 Matrix{Float64}:
  0.5          0.0  0.0
  0.0          0.5  0.0
 -6.93889e-18  0.0  0.5

In [15]:
dN_dx = transpose(dNdX) / Je

8×3 Matrix{Float64}:
 -0.166667   -0.166667   -0.622008
  0.166667   -0.0446582  -0.166667
  0.0446582   0.0446582  -0.0446582
 -0.0446582   0.166667   -0.166667
 -0.622008   -0.622008    0.622008
  0.622008   -0.166667    0.166667
  0.166667    0.166667    0.0446582
 -0.166667    0.622008    0.166667

In [16]:
using Kronecker    
Bu = zeros(9, 24);
for k=1:8
    Bu[:, (k-1)*3+1:k*3] = kronecker(I(3), dN_dx[k,:]);
end
Bu

9×24 Matrix{Float64}:
 -0.166667  -0.0       -0.0       …  -0.166667  -0.0       -0.0
 -0.166667  -0.0       -0.0           0.622008   0.0        0.0
 -0.622008  -0.0       -0.0           0.166667   0.0        0.0
 -0.0       -0.166667  -0.0          -0.0       -0.166667  -0.0
 -0.0       -0.166667  -0.0           0.0        0.622008   0.0
 -0.0       -0.622008  -0.0       …   0.0        0.166667   0.0
 -0.0       -0.0       -0.166667     -0.0       -0.0       -0.166667
 -0.0       -0.0       -0.166667      0.0        0.0        0.622008
 -0.0       -0.0       -0.622008      0.0        0.0        0.166667

In [17]:
dudX = Bu * d

9-element Vector{Float64}:
 -0.0012110899604627278
 -0.004709544116449797
 -1.0842021724855044e-19
  0.004706716693526172
 -0.0006110966224333662
  2.710505431213761e-20
  5.2150124496552763e-17
  6.16911036144252e-17
  0.0005746657217790565

In [18]:
Fdef = I(3) + transpose(reshape(dudX, (3, 3)))

3×3 Matrix{Float64}:
 0.998789     -0.00470954   -1.0842e-19
 0.00470672    0.999389      2.71051e-20
 5.21501e-17   6.16911e-17   1.00057

In [19]:
bdef = Fdef  * transpose(Fdef)

3×3 Matrix{Float64}:
  0.997601     -5.64969e-6  5.16879e-17
 -5.64969e-6    0.9988      6.1926e-17
  5.16879e-17   6.1926e-17  1.00115

In [20]:
estrain = (I - inv(bdef)) / 2

3×3 Matrix{Float64}:
 -0.00120215   -2.83504e-6   2.58765e-17
 -2.83504e-6   -0.000600554  3.09647e-17
  2.58765e-17   3.09647e-17  0.000574171

In [21]:
Fp_n = I(3)    
Fe_tr = Fdef / Fp_n


3×3 Matrix{Float64}:
 0.998789     -0.00470954   -1.0842e-19
 0.00470672    0.999389      2.71051e-20
 5.21501e-17   6.16911e-17   1.00057

In [22]:
be_tr = Fe_tr * transpose(Fe_tr)

3×3 Matrix{Float64}:
  0.997601     -5.64969e-6  5.16879e-17
 -5.64969e-6    0.9988      6.1926e-17
  5.16879e-17   6.1926e-17  1.00115

In [23]:
D = eigvals(be_tr)

3-element Vector{Float64}:
 0.99760144
 0.9988003600000002
 1.00114966168425

In [24]:
lambda_e_tr = sqrt.(D)

3-element Vector{Float64}:
 0.9988
 0.9994000000000001
 1.000574665721779

In [25]:
Jdef_e_tr = prod(lambda_e_tr)

0.9987743517372393

In [26]:
tau_princ_coef(J, eigval) = lambda * log(J) - mu + mu * eigval * eigval

tau_princ_coef (generic function with 1 method)

In [27]:
tau_princ_tr = zeros(3)
for i=1:3
    tau_princ_tr[i] = tau_princ_coef(Jdef_e_tr, lambda_e_tr[i]);
end
tau_princ_tr

3-element Vector{Float64}:
   -5.677295975828886e6
   -3.759023975828886e6
 -141.2810287475586

In [28]:
function coaxial(A, b)
    eig_vec = eigvecs(A);
    C       = zeros(3, 3);
    for i=1:3
        C = C + b[i] * reshape(kron(eig_vec[:, i], eig_vec[:, i]), (3,3))
    end
    return C
end

coaxial (generic function with 1 method)

In [29]:
f = coaxial(be_tr, tau_princ_tr)

3×3 Matrix{Float64}:
    -5.67725e6  -9039.51          0.0
 -9039.51          -3.75907e6     0.0
     0.0            0.0        -141.281

In [30]:
tau_tr = coaxial(be_tr, tau_princ_tr)

3×3 Matrix{Float64}:
    -5.67725e6  -9039.51          0.0
 -9039.51          -3.75907e6     0.0
     0.0            0.0        -141.281

In [31]:
p_tau_tr = sum(tau_princ_tr) / 3
dev_tau_princ_tr = tau_princ_tr .- p_tau_tr

3-element Vector{Float64}:
      -2.5318088982667127e6
 -613536.8982667127
       3.145345796533426e6

In [32]:
ve_tr = sqrt(be_tr)

3×3 Matrix{Float64}:
  0.9988      -2.82739e-6  0.0
 -2.82739e-6   0.9994      0.0
  0.0          0.0         1.00057

In [33]:
Re_tr = inv(ve_tr) * Fe_tr

3×3 Matrix{Float64}:
 0.999989     -0.00471237   -1.0855e-19
 0.00471237    0.999989      2.7121e-20
 5.21202e-17   6.16557e-17   1.0

In [34]:
c_tau_n = 2.5e6
c_tau           = c_tau_n

2.5e6

In [35]:
dev_tau_tr_norm = norm(dev_tau_princ_tr)

4.084076885006909e6

In [36]:
f_tr = dev_tau_tr_norm - Aphi * c_tau_n + Bphi * p_tau_tr

1576.885006908793

In [37]:
dgdtau_princ_tr = dev_tau_princ_tr ./ dev_tau_tr_norm .+ Bpsi / 3

3-element Vector{Float64}:
 -0.6199219479832172
 -0.15022657886757065
  0.7701485268507879

In [38]:
trace_dgdtau_tr = sum(dgdtau_princ_tr)


1.1102230246251565e-16

In [39]:
dgdtau_tr       = coaxial(be_tr, dgdtau_princ_tr)

3×3 Matrix{Float64}:
 -0.619912    -0.00221335  0.0
 -0.00221335  -0.150237    0.0
  0.0          0.0         0.770149

In [42]:
f_tr

1576.885006908793

In [45]:
lambda_e = lambda_e_tr
Jdef_e   = prod(lambda_e)

0.9987743517372393

In [50]:
tau_princ = zeros(3)
for i=1:3
    tau_princ[i] = tau_princ_coef(Jdef_e, lambda_e[i])
end
p_tau         = sum(tau_princ) / 3
dev_tau_princ = tau_princ .- p_tau
dev_tau_norm  = norm(dev_tau_princ)

4.084076885006909e6

In [54]:
        fyield_atol      = 1e-6
        fyield_rtol      = 1e-8
        k                = 0
        iter_break_local = 10
        fyield           = f_tr
        Dg_iter          = 0

0

In [58]:
# derivative of devtau wrt Dg
dtau_princ_dDg = zeros(3)
for i=1:3
    dtau_princ_dDg[i] = -lambda * trace_dgdtau_tr - 2 * mu * lambda_e[i] * lambda_e[i] * dgdtau_princ_tr[i]
end
trace_dtaudDg      = sum(dtau_princ_dDg)
dev_dtau_princ_dDg = dtau_princ_dDg .- trace_dtaudDg / 3

3-element Vector{Float64}:
  1.9817148075094583e9
  4.828710732977314e8
 -2.46458588080719e9

In [62]:
ddevtau_norm_dDg = dot(dev_tau_princ, dev_dtau_princ_dDg) / dev_tau_norm
dfdDg = ddevtau_norm_dDg - Hc * Aphi * Apsi + Bphi * trace_dtaudDg / 3

-3.1991465751084943e9

In [66]:
del_Dg  = -fyield / dfdDg

4.92908020900954e-7

In [67]:
Dg_iter = Dg_iter + del_Dg


4.92908020900954e-7

In [68]:
if Dg_iter < 0
    Dg_iter
end

In [89]:
for i=1:3
    lambda_e[i] = lambda_e_tr[i] * exp(-Dg_iter * dgdtau_princ_tr[i])
end
lambda_e
Jdef_e = prod(lambda_e)

0.9987743517372403

In [93]:
for i=1:3
    tau_princ[i] = lambda * log(Jdef_e) - mu + mu * lambda_e[i] * lambda_e[i]
end
tau_princ

3-element Vector{Float64}:
     -5.667541335276842e6
     -3.7566572843165398e6
 -12302.79684472084

In [94]:
p_tau        = sum(tau_princ) / 3
dev_tau      = tau_princ .- p_tau
dev_tau_norm = norm(dev_tau)

4.0683080557988184e6

In [95]:
c_tau = c_tau_n + Dg_iter * Hc * Apsi

2.500000000246458e6

In [96]:
fyield = dev_tau_norm - Aphi * c_tau + Bphi * p_tau

-14191.944603648037

In [97]:
dev_tau

3-element Vector{Float64}:
      -2.522040863130808e6
 -611156.8121705055
       3.1331976753013134e6

In [98]:
tau_princ

3-element Vector{Float64}:
     -5.667541335276842e6
     -3.7566572843165398e6
 -12302.79684472084

In [99]:
p_tau

-3.1455004721460342e6

In [101]:
Fp = transpose(Re_tr) * exp(Dg_iter * dgdtau_tr) * Re_tr * Fp_n

3×3 Matrix{Float64}:
  1.0          -2.78423e-16  5.20118e-17
 -2.7929e-16    1.0          6.16833e-17
  5.20118e-17   6.16833e-17  1.0

In [105]:
tau = coaxial(be_tr, tau_princ)

3×3 Matrix{Float64}:
    -5.6675e6  -9004.7            0.0
 -9004.7          -3.7567e6       0.0
     0.0           0.0       -12302.8

In [106]:
println(k)
                

0


In [108]:
println("reached max number of local iterations for Dg")

reached max number of local iterations for Dg


In [109]:
fyield

-14191.944603648037

In [111]:
abs(fyield)/fyield_atol

1.4191944603648037e10